In [ ]:
pip install sentence-transformers

In [ ]:
pip install torch

In [ ]:
pip install GPUtil

In [4]:
import torch
import psutil
import time


In [5]:
import GPUtil

In [6]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax

In [7]:
import pandas as pd
import numpy as np

In [ ]:
# from sentence_transformers import SentenceTransformer, util

In [9]:
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
#model.save_pretrained(MODEL)

In [11]:
data = pd.read_csv("/content/Diff_Prod_Reviews.csv")

In [12]:
data[:5]

,Name,Product Name,Stars,Title,Date,Description
0,Chandan Bhagat,Nike Women's WMNS Revolution 4 Obsdn/Mountain ...,1.0,1.0 out of 5 stars\nBad quality,11-04-2024,"Shoes sole came out in 4th day of usage, very ..."
1,akshay sharma,Nike Women's WMNS Revolution 4 Obsdn/Mountain ...,2.0,2.0 out of 5 stars\nQuality issues.,20-10-2019,Have to return it because of quality issues al...
2,Amazon Customer,Nike Women's WMNS Revolution 4 Obsdn/Mountain ...,2.0,2.0 out of 5 stars\nNot comfortable,14-06-2019,Not satisfying
3,Himanshu Yadav,Nike Women's WMNS Revolution 4 Obsdn/Mountain ...,1.0,1.0 out of 5 stars\nFake nike shoe,25-10-2020,The product was not found original and branded...
4,Pooja,Nike Women's WMNS Revolution 4 Obsdn/Mountain ...,1.0,1.0 out of 5 stars\nTorn shoes,13-07-2020,The box was damaged and the shoe was torn


In [13]:
length = len(data)

In [14]:
length

1082

In [ ]:
# Load a BERT model
# topic_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [ ]:
# List of words
# word_list = ["Fit & Size", "Usage", "Feel", "Caregiver", "Activity", "Aesthetics", "Customer Service", "Protection", "Absorbency", "Medical Reco", "Packaging", "Cost", "Value", "Promo"]

In [16]:
# Start time
start_time = time.time()

for i in range(length):
  text = str(data.iloc[i,5])
  encoded_input = tokenizer(text, return_tensors='pt')
  output = model(**encoded_input)
  scores = output[0][0].detach().numpy()
  scores = softmax(scores)
  ranking = np.argsort(scores)

  data.loc[i,'Negative_Score'] = scores[0]
  data.loc[i,'Neutral_Score'] = scores[1]
  data.loc[i,'Positive_Score'] = scores[2]

  if ranking[2] == 0:
    data.loc[i,'Model_Sentiment'] = 'Negative'
  elif ranking[2] == 1:
    data.loc[i,'Model_Sentiment'] = 'Neutral'
  elif ranking[2] == 2:
    data.loc[i,'Model_Sentiment'] = 'Positive'
  else:
    data.loc[i,'Model_Sentiment'] = 'None'

  # End time
end_time = time.time()


In [17]:
data

,Name,Product Name,Stars,Title,Date,Description,Negative_Score,Neutral_Score,Positive_Score,Model_Sentiment
0,Chandan Bhagat,Nike Women's WMNS Revolution 4 Obsdn/Mountain ...,1.0,1.0 out of 5 stars\nBad quality,11-04-2024,"Shoes sole came out in 4th day of usage, very ...",0.936720,0.056717,0.006564,Negative
1,akshay sharma,Nike Women's WMNS Revolution 4 Obsdn/Mountain ...,2.0,2.0 out of 5 stars\nQuality issues.,20-10-2019,Have to return it because of quality issues al...,0.874939,0.117490,0.007571,Negative
2,Amazon Customer,Nike Women's WMNS Revolution 4 Obsdn/Mountain ...,2.0,2.0 out of 5 stars\nNot comfortable,14-06-2019,Not satisfying,0.721651,0.223401,0.054948,Negative
3,Himanshu Yadav,Nike Women's WMNS Revolution 4 Obsdn/Mountain ...,1.0,1.0 out of 5 stars\nFake nike shoe,25-10-2020,The product was not found original and branded...,0.575365,0.404719,0.019916,Negative
4,Pooja,Nike Women's WMNS Revolution 4 Obsdn/Mountain ...,1.0,1.0 out of 5 stars\nTorn shoes,13-07-2020,The box was damaged and the shoe was torn,0.843265,0.147426,0.009308,Negative
...,...,...,...,...,...,...,...,...,...,...
1077,Shanmugaraj,boAt BassHeads 100 in-Ear Wired Headphones wit...,2.0,2.0 out of 5 stars\nBuds are not fixed well,28-01-2024,"After using this headset, the buds are stuck i...",0.717453,0.259458,0.023089,Negative
1078,Yashik,boAt BassHeads 100 in-Ear Wired Headphones wit...,2.0,2.0 out of 5 stars\nWorst music quality. Too m...,17-11-2023,Here is an honest review. Too much hype in rat...,0.779119,0.181986,0.038895,Negative
1079,Balaji M,boAt BassHeads 100 in-Ear Wired Headphones wit...,2.0,2.0 out of 5 stars\nMisfit,16-02-2024,Very poor quality. The jack seems to be a misf...,0.936512,0.057451,0.006037,Negative
1080,Aravinda S.,boAt BassHeads 100 in-Ear Wired Headphones wit...,3.0,3.0 out of 5 stars\nBoat wired Earphones,16-11-2023,The media could not be loaded.\n ...,0.611520,0.353163,0.035317,Negative


In [27]:
data['Model_Sentiment'].value_counts()

Model_Sentiment
Negative    647
Neutral     234
Positive    201
Name: count, dtype: int64

In [39]:
for i in data['Product Name'].unique():
  print("\nValue counts for ", i)
  # data[data['Product Name'].str.contains('Nike')]['Model_Sentiment'].value_counts()
  print(data[data['Product Name'] == i]['Model_Sentiment'].value_counts())
  print("Mean for star rating of ",i)
  print(round(np.mean(data[data['Product Name'] == i]['Stars']),0))


Value counts for  Nike Women's WMNS Revolution 4 Obsdn/Mountain Blue-T.Blue Running...
Model_Sentiment
Negative    24
Positive     5
Neutral      4
Name: count, dtype: int64
Mean for star rating of  Nike Women's WMNS Revolution 4 Obsdn/Mountain Blue-T.Blue Running...
1.0

Value counts for  Apple iPhone 15 (128 GB) - Black
Model_Sentiment
Negative    358
Neutral      91
Positive     50
Name: count, dtype: int64
Mean for star rating of  Apple iPhone 15 (128 GB) - Black
1.0

Value counts for  boAt BassHeads 100 in-Ear Wired Headphones with Mic (Black)
Model_Sentiment
Negative    265
Positive    146
Neutral     139
Name: count, dtype: int64
Mean for star rating of  boAt BassHeads 100 in-Ear Wired Headphones with Mic (Black)
3.0


In [33]:
data['Stars'].dtype

dtype('float64')

1.2857142857142858

In [18]:
data.to_csv("Amz_Scrapped_Sentiment.csv",index=False)

In [22]:
# GPU stats
gpu_usage = torch.cuda.memory_allocated()
gpu_memory_allocated = torch.cuda.memory_allocated()
gpu_memory_cached = torch.cuda.memory_reserved()

# CPU stats
cpu_usage = psutil.cpu_percent()
cpu_memory_usage = psutil.virtual_memory().percent

# Time taken
time_taken = end_time - start_time


# Print stats
print(f"GPU Usage: {gpu_usage} bytes")
print(f"GPU Memory Allocated: {gpu_memory_allocated} bytes")
print(f"GPU Memory Cached: {gpu_memory_cached} bytes")
print(f"CPU Usage: {cpu_usage}%")
print(f"CPU Memory Usage: {cpu_memory_usage}%")
print(f"Time Taken: {time_taken} seconds")

GPU Usage: 0 bytes
GPU Memory Allocated: 0 bytes
GPU Memory Cached: 0 bytes
CPU Usage: 27.8%
CPU Memory Usage: 21.4%
Time Taken: 600.4442734718323 seconds


In [23]:
# Get GPU information
gpus = GPUtil.getGPUs()
if gpus:
    gpu = gpus[0]  # Assuming a single GPU is used
    print(f"GPU Name: {gpu.name}, GPU Memory Usage: {gpu.memoryUsed} MB")


GPU Name: Tesla T4, GPU Memory Usage: 0.0 MB


In [24]:
import subprocess

def get_gpu_info():
    try:
        result = subprocess.run(['nvidia-smi', '--query-gpu=name,driver_version', '--format=csv,noheader'], stdout=subprocess.PIPE)
        gpu_info = result.stdout.decode('utf-8').strip().split('\n')
        return gpu_info
    except Exception as e:
        print(f"Error getting GPU information: {e}")
        return None

In [25]:
gpu_info = get_gpu_info()
if gpu_info:
    for i, info in enumerate(gpu_info):
        gpu_name, driver_version = info.split(',')
        print(f"GPU {i+1}: Name - {gpu_name.strip()}, Driver Version - {driver_version.strip()}")
else:
    print("GPU information not available.")

GPU 1: Name - Tesla T4, Driver Version - 535.104.05
